In [1]:
import bw2data as bd
import bw2io as bi
import bw2calc as bc
import pandas as pd
from scipy.special import expit
from matplotlib import pyplot as plt
import numpy as np
import bw_processing as bwp

In [2]:
def replacement(cf_mass):
    return 5 - cf_mass * (1 + expit(cf_mass * 4 - 5)) + np.random.rand()

In [4]:
class MyCoolAluminiumThing:
    def __next__(self):
        cf = np.random.rand() * 2.5
        alu = replacement(cf)
        return np.array([cf, alu])

In [26]:
my_interface = MyCoolAluminiumThing()
next(my_interface)

array([1.36071002, 3.59916079])

In [5]:
bd.projects.set_current("fresh-start")

In [6]:
if "mobility" in bd.databases:
    del bd.databases['mobility']
    
if "biosphere" in bd.databases:
    del bd.databases['biosphere']    

In [7]:
db = bd.Database("mobility")
db.register()

biosphere = bd.Database("biosphere")
biosphere.register()

In [8]:
data = {
    'code': 'bike',
    'name': 'bike production',
    'location': 'NO',
    'unit': 'bike'
}

bike = db.new_activity(**data)
bike.save()

In [9]:
data = {
    'code': 'CF',
    'name': 'carbon fibre',
    'unit': 'kilogram',
    'location': 'CN'
}

cf = db.new_activity(**data)
cf.save()

In [10]:
data = {
    'code': 'Alu',
    'name': 'Aluminium',
    'unit': 'kilogram',
    'location': 'NO'
}

alu = db.new_activity(**data)
alu.save()

In [11]:
ng = db.new_activity(
    name="Nat Gas", 
    code='ng', 
    location='NO', 
    unit='MJ'
)

ng.save()

In [12]:
co2 = biosphere.new_activity(
    name="Carbon Dioxide", 
    code='co2', 
    categories=('air',),
    type='emission',
)

co2.save()

In [13]:
bike.new_exchange(
    amount=2.5, 
    type='technosphere',
    input=cf
).save()

In [14]:
bike.new_exchange(
    amount=0, 
    type='technosphere',
    input=alu,
).save()

In [15]:
cf.new_exchange(
    amount=237.3,  # plus 58 kWh of electricity, in ecoinvent 3.8 
    uncertainty_type=5, 
    minimum=200, 
    maximum=300, 
    type='technosphere',
    input=ng,
).save()

In [16]:
ng.new_exchange(
    amount=26.6 / 237, 
    uncertainty_type=5, 
    minimum=26 / 237,
    maximum=27.2 / 237, 
    type='biosphere',
    input=co2,
).save()

In [17]:
alu.new_exchange(
    amount=8, 
    uncertainty_type=2, 
    loc=8,
    scale=1,
    type='biosphere',
    input=co2,
).save()

In [18]:
ipcc = bd.Method(('IPCC',))
ipcc.write([
    (co2.key, {'amount': 1, 'uncertainty_type': 3, 'loc': 1, 'scale': 0.05}),
])

In [19]:
import bw2calc as bc

What if we only want uncertainty in the characterization step?

We need to modify the datapackage a bit

In [20]:
from bw_processing import load_datapackage
from fs.zipfs import ZipFS

In [21]:
database_dp = load_datapackage(ZipFS(db.filepath_processed()))
method_dp = load_datapackage(ZipFS(ipcc.filepath_processed()))

We can use the datapackages directly, just as before:

In [22]:
interface_dp = bwp.create_datapackage()

In [23]:
indices_array = np.array([(cf.id, bike.id), (alu.id, bike.id)], dtype=bwp.INDICES_DTYPE)
indices_array

array([(2651, 2650), (2652, 2650)], dtype=[('row', '<i4'), ('col', '<i4')])

In [24]:
{o.id: o for o in db}

{2651: 'carbon fibre' (kilogram, CN, None),
 2652: 'Aluminium' (kilogram, NO, None),
 2653: 'Nat Gas' (MJ, NO, None),
 2650: 'bike production' (bike, NO, None)}

In [25]:
interface_dp.add_dynamic_vector?

Signature:
interface_dp.add_dynamic_vector(
    *,
    matrix: str,
    interface: Any,
    indices_array: numpy.ndarray,
    name: Optional[str] = None,
    flip_array: Optional[numpy.ndarray] = None,
    keep_proxy: bool = False,
    **kwargs,
) -> None
Docstring: <no docstring>
File:      ~/miniconda3/envs/bw25rc/lib/python3.9/site-packages/bw_processing/datapackage.py
Type:      method


In [27]:
interface_dp.add_dynamic_vector(
    matrix="technosphere_matrix",
    interface=my_interface,
    name="alu replacing CF interface",
    flip_array=np.array([True, True]),
    indices_array=indices_array,
)    

In [28]:
lca = bc.LCA(
    demand={bike.id: 1},
    data_objs=[database_dp, method_dp, interface_dp],
    use_distributions=False,
    # use_arrays=True,
    seed_override=None
)
lca.lci()
lca.lcia()

In [29]:
pd.DataFrame([
    {
        'score': lca.score, 
        'inv': lca.inventory.sum(), 
        'char': lca.characterization_matrix.sum()
    } for _, _ in zip(lca, range(10))
])

,score,inv,char
0,66.991851,66.991851,1.0
1,64.444448,64.444448,1.0
2,52.687967,52.687967,1.0
3,61.382886,61.382886,1.0
4,72.727043,72.727043,1.0
5,68.091807,68.091807,1.0
6,61.560548,61.560548,1.0
7,59.417452,59.417452,1.0
8,48.682113,48.682113,1.0
9,59.373211,59.373211,1.0


In [34]:
lca.technosphere_matrix.toarray()

array([[   1.        ,    0.        ,    0.        ,    0.        ],
       [  -1.22493402,    1.        ,    0.        ,    0.        ],
       [  -3.58521285,    0.        ,    1.        ,    0.        ],
       [   0.        , -237.30000305,    0.        ,    1.        ]])

In [35]:
next(lca)